In [1]:
# 나이브 베이즈 분류기 테스트를 위한 코드

In [1]:
import json
import random
import tqdm
import re
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

from konlpy.tag import Mecab, Okt

In [2]:
# 필요 데이터 로드
df = pd.read_csv('raw.csv')

with open('data.json','r') as f:
    dataset = json.load(f)
    
with open('valid_data.json','r') as f:
    valid_dataset = json.load(f)

### TF-IDF 다시 정의해 나이브 베이즈 대입하는 과정

In [19]:
#형태소 분석기
flag = 0

try:
    Ko_Analyzer = Mecab(dicpath = r"C:/mecab/mecab-ko-dic")
    flag = 1
except:
    Ko_Analyzer = Okt()

In [10]:
flag = 0
Ko_Analyzer = Okt()

In [20]:
# 텍스트 전처리 관련 함수 정의
def text_preprocessing(text):
    # 불용어(Stopword)는 공통된 단어 혹은 비교 기준 명사로 쓰일 수 없는 것들을 사용한다.
    stop_words = ['은', '는', '이', '가', '하', '아', '것', '들', '의','번','장','방','시','에서','우리',
                  '년', '있', '되', '수', '보', '주', '등', '한', '및','면','뿐','나','그것','이것','때문',
                  '마찬가지','곳','로','와','만','데','한때','이후','이전','지금','때','각종','한편','대부분',
                  '를','앞','후','경우','간','반면','채','어젠','서','해당','과','외','예','중','사','조','항','제',
                  '뒤','현재','무엇','중','그','명','예로','군데','내','너','여기','만큼','동안','듯','을','그','개',
                  '지','여도','덕분','최근', '밖', '이상', '처음', '최소','당시','결국',
                 # 여기서 부터는 공통 명사
                  '사회', '필요', '지원', '관련', '정책', '정부', '결과', '연구', '분석', '국가', '문제',
                  '이번', '북한', '기술', '기업', '정보'
                 ]
    
    # 한글 및 공백을 제외한 문자 제거(영문자, 특수문자)
    text = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ]", " ", text)
    temp = []
    
    # Mecab 형태소 분석기를 사용한 품사 태깅
    if flag == 1:
        pos_data = Ko_Analyzer.pos(text)
        # 품사 태깅한 것들 중 일반 명사, 지정 명사 만을 사용
        for i in pos_data:
            if i[1] == 'NNG' or i[1] == 'NNP':
                temp.append(i[0])
            else : continue
    else:
        pos_data = Ko_Analyzer.pos(text, norm = True)
        for i in pos_data:
            if i[1] == 'Noun':
                temp.append(i[0])
            else : continue
    
    
    #nouns = mecab.nouns(text)  # 명사 추출
    
    # 명사 중 불용어에 해당되면 삭제
    nouns = [token for token in temp if not token in stop_words]
    
    return list(nouns)

In [6]:
# label 리스트 화 및 중복 제거 과정
label_list = list(set(dataset['label']))

In [7]:
len(label_list)

99

In [8]:
# 라벨별 content 데이터 셋 초기화
label_dataset = {'label' : [], 'content' : []}

# doc_name_list에 따른 문서 제목, 내용 입력
for i in label_list:
    label_dataset['label'].append(i)
    temp = list(df.loc[df['kdc_label'] == i]['passage'].values[:])
    temp = " ".join(temp)
    label_dataset['content'].append(temp)

In [ ]:
# DTM with default tokenizer
dtmvector = CountVectorizer(ngram_range = (1,1))
X_train_dtm = dtmvector.fit_transform(label_dataset['content'])

# TF-IDF 생성
tfidf_transformer = TfidfTransformer()
tfidffv = tfidf_transformer.fit_transform(X_train_dtm)
print(tfidffv.shape)

In [29]:
# DTM with custom tokenizer
dtmvector = CountVectorizer(tokenizer = text_preprocessing, ngram_range = (2,3))
X_train_dtm = dtmvector.fit_transform(label_dataset['content'])

print(X_train_dtm.shape)

(99, 12447922)


In [30]:
# TF-IDF 생성
tfidf_transformer = TfidfTransformer()
tfidffv = tfidf_transformer.fit_transform(X_train_dtm)
print(tfidffv.shape)

(99, 12447922)


In [ ]:
# 다항 나이브 베이즈 분류기(alpha = 0.5)
mod_2 = MultinomialNB(alpha = 0.5, class_prior = None, fit_prior = True)
mod_2.fit(tfidffv, label_dataset['label']) # fit : X, Y에 맞춰서 분류기를 생성

In [44]:
# 다항 나이브 베이즈 분류기(alpha = 1.0)
mod = MultinomialNB(alpha = 1.0, class_prior = None, fit_prior = True)
mod.fit(tfidffv, label_dataset['label']) # fit : X, Y에 맞춰서 분류기를 생성

MultinomialNB()

In [42]:
## test dataset 만들기..중복없이 500개의 랜덤 샘플 생성
random_idx = []
random_num = random.randint(0,3998)

for i in range(500):
    while random_num in random_idx:
        random_num = random.randint(0,3998)
    random_idx.append(random_num)

test_dataset = {'label' : [], 'content' : []}

# 랜덤 인덱스에 따른 새로운 test dataset 생성
for i in random_idx:
    test_dataset['label'].append(valid_dataset['label'][i])
    test_dataset['content'].append(valid_dataset['content'][i])
    
#print(test_dataset['label'][0])
#print(test_dataset['content'][0])

In [46]:
%%time
# time을 사용해 해당 셀(분류기 예측)에 대한 성능 및 시간 테스트
# Test dataset에 대한 tf-idf 생성 과정
X_test_dtm = dtmvector.transform(test_dataset['content'])

print(X_test_dtm.shape)

tfidfv_test = tfidf_transformer.transform(X_test_dtm)
print(tfidfv_test.shape)

# 예측
predicted = mod.predict(tfidfv_test)
print("정확도:", accuracy_score(test_dataset['label'],predicted))

#print(predicted)
#print(test_dataset['label'])

(500, 12447922)
(500, 12447922)
정확도: 0.922
Wall time: 1min 18s


In [ ]:
%%time
# time을 사용해 해당 셀(분류기 예측)에 대한 성능 및 시간 테스트
# Test dataset에 대한 tf-idf 생성 과정
X_test_dtm = dtmvector.transform(test_dataset['content'])

print(X_test_dtm.shape)

tfidfv_test = tfidf_transformer.transform(X_test_dtm)
print(tfidfv_test.shape)

# 예측
predicted = mod_2.predict(tfidfv_test)
print("정확도:", accuracy_score(test_dataset['label'],predicted))

#print(predicted)
#print(test_dataset['label'])

In [ ]:
'''
############ BY VALID ############
<with custom tokenizer / alpha = 1.0>
ngram = 1,1 : 0.542 / 0.564 / 0.538 / 0.568 / 0.548
ngram = 1,2 : 0.626 / 0.632 / 0.596 / 0.66 / 0.626
ngram = 1,3 : 0.72 / 0.694 / 0.694 / 0.694 / 
ngram = 2,3 : 0.906 / 0.902 / 0.914 / 0.914 / 0.932 (약 1분 소요)

<with custom tokenizer / alpha = 0.5>
ngram = 1,1 : 0.542 / 0.564 / 0.538 / 0.568 / 0.548
ngram = 1,2 : 0.626 / 0.632 / 0.596 / 0.66 / 0.626
ngram = 1,3 : 0.72 / 0.694 / 0.694 / 0.694 / 
ngram = 2,3 : 0.910 / 0.928 / 0.916 / 0.92 / 0.922 (약 1분 소요) (vector : 12447922)

<with default tokenizer> : 내장된 토크나이징 기법으로는 너무 많은 벡터 생성 -> 처리 시간 느림
ngram = 1,1 : 0.73
ngram = 1,2 : 0.864
ngram = 1,3 : 측정불가
ngram = 2,3 : 측정불가

'''